In [14]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:

import src.config as config
from src.feature_store_api import get_feature_store, get_or_create_feature_view
from src.config import FEATURE_VIEW_METADATA
from src.data import add_missing_slots
from src.inference import load_batch_of_features_from_store
from datetime import datetime, timedelta
import pandas as pd


current_date = pd.to_datetime(datetime.utcnow()).floor('H') # - timedelta(hours=1)
print(f'{current_date=}')

# feature_view = get_or_create_feature_view(FEATURE_VIEW_METADATA)

# # fetch data from the feature store
# fetch_data_from = current_date - timedelta(days=28)
# fetch_data_to = current_date

# # add plus minus margin to make sure we do not drop any observation
# ts_data = feature_view.get_batch_data(
#     start_time=fetch_data_from - timedelta(days=1),
#     end_time=fetch_data_to + timedelta(days=1)
# )
# ts_data

current_date=Timestamp('2024-02-29 03:00:00')


In [16]:

# # filter data to the time period we are interested in
# pickup_ts_from = int(fetch_data_from.timestamp() * 1000)
# pickup_ts_to = int(fetch_data_to.timestamp() * 1000)
# ts_data = ts_data[ts_data.pickup_ts.between(pickup_ts_from, pickup_ts_to)]


In [17]:

# ts_data = add_missing_slots(ts_data)


In [18]:
# len(ts_data)

In [19]:
# location_ids = ts_data['pickup_location_id'].unique()

# config.N_FEATURES * len(location_ids)

In [20]:
# current_date = pd.Timestamp('2023-02-28 09:00:00')
features = load_batch_of_features_from_store(current_date)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/423065
Connected. Call `.close()` to terminate connection gracefully.
2024-02-28 19:36:38,147 INFO: Feature view already exists, skipping creation.
Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/423065
Connected. Call `.close()` to terminate connection gracefully.
Finished: Reading data from Hopsworks, using ArrowFlight (6.17s) 


100%|██████████| 265/265 [00:01<00:00, 256.86it/s]


In [21]:
from src.inference import (
    load_model_from_registry,
    get_model_predictions
)

model = load_model_from_registry()
predictions = get_model_predictions(model, features)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/423065
Connected. Call `.close()` to terminate connection gracefully.
[LightGBM] [Warning] bagging_fraction is set=0.24794648125877955, subsample=1.0 will be ignored. Current value: bagging_fraction=0.24794648125877955


See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)


In [22]:
predictions['pickup_hour'] = current_date
predictions

,pickup_location_id,predicted_demand,pickup_hour
0,1,8.0,2024-02-29 03:00:00
1,2,10.0,2024-02-29 03:00:00
2,3,10.0,2024-02-29 03:00:00
3,4,13.0,2024-02-29 03:00:00
4,5,10.0,2024-02-29 03:00:00
...,...,...,...
260,261,9.0,2024-02-29 03:00:00
261,262,4.0,2024-02-29 03:00:00
262,263,6.0,2024-02-29 03:00:00
263,264,13.0,2024-02-29 03:00:00


### Save these predictions in the feature store, so they can be later consumed by our Streamlit app

In [23]:
from src.feature_store_api import get_feature_store
import src.config as config

# connect to the feature group
feature_group = get_feature_store().get_or_create_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTIONS,
    version=1,
    description="Predictions generate by our production model",
    primary_key = ['pickup_location_id', 'pickup_hour'],
    event_time='pickup_hour',
)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/423065
Connected. Call `.close()` to terminate connection gracefully.


In [24]:
feature_group.insert(predictions, write_options={"wait_for_job": True})

Uploading Dataframe: 0.00% |          | Rows 0/265 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: model_predictions_feature_group_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/423065/jobs/named/model_predictions_feature_group_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x133661e8370>, None)